In [6]:
import run_simulation as rs
import numpy as np	
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tdgl
import os 


In [7]:
rs.check_system()

GPU is available
GPU:  b'NVIDIA GeForce RTX 4060 Laptop GPU'


In [8]:
xi=0.1
size=1.0

min_c=0
max_c=50
num_c=100
H_field_range=np.linspace(0,25,20)
current_range=np.linspace(min_c,max_c,num_c)
device=rs.define_4_terminal_mesh(size,size,xi=xi, probes=[((-size*(0.4),0),
					   				(size*(0.4),0),)],
									terminal_width=size/30,
									terminal_height=size/10
									)
z=np.zeros((10,10))

In [10]:
folder_path = 'H_fields_4'
files = rs.get_all_files(folder_path)

data_list=[]

for file in files:
	H_field, current = rs.extract_numbers(file)
	array=np.load(file)
	data=[H_field,current,array]
	data_list.append(data)
	

In [12]:
H_fields_list=[]
currents_for_H=[]
currents=[]


for data in data_list:
	if data[0] not in H_fields_list:
		H_fields_list.append(data[0])
		currents_for_H.append(currents)
		currents=[data[1]]
	else:
		index=H_fields_list.index(data[0])
		currents.append(data[1])
		
currents_for_H.append(currents)
currents_for_H=currents_for_H[1:]
	

found_Hs,missing_Hs=rs.check_elements_in_list(H_field_range,H_fields_list)

print(f"found data for H= {found_Hs}")
print(f"missing data for H= {missing_Hs}")

simulations_to_be_computed={}

for ii in range(len(found_Hs)):
	found_C,missing_C=rs.check_elements_in_list(current_range,currents_for_H[ii])

	simulations_to_be_computed[found_Hs[ii]]=np.array(missing_C)

for ii in range(len(missing_Hs)):

	simulations_to_be_computed[missing_Hs[ii]]=current_range

for key, value in simulations_to_be_computed.items():
    print(f"{key}: {value}")

found data for H= []
missing data for H= [0.0, 1.3157894736842106, 2.6315789473684212, 3.947368421052632, 5.2631578947368425, 6.578947368421053, 7.894736842105264, 9.210526315789474, 10.526315789473685, 11.842105263157896, 13.157894736842106, 14.473684210526317, 15.789473684210527, 17.10526315789474, 18.42105263157895, 19.736842105263158, 21.05263157894737, 22.368421052631582, 23.68421052631579, 25.0]
0.0: [ 0.          0.50505051  1.01010101  1.51515152  2.02020202  2.52525253
  3.03030303  3.53535354  4.04040404  4.54545455  5.05050505  5.55555556
  6.06060606  6.56565657  7.07070707  7.57575758  8.08080808  8.58585859
  9.09090909  9.5959596  10.1010101  10.60606061 11.11111111 11.61616162
 12.12121212 12.62626263 13.13131313 13.63636364 14.14141414 14.64646465
 15.15151515 15.65656566 16.16161616 16.66666667 17.17171717 17.67676768
 18.18181818 18.68686869 19.19191919 19.6969697  20.2020202  20.70707071
 21.21212121 21.71717172 22.22222222 22.72727273 23.23232323 23.73737374
 24.24

In [13]:
rs.build_mesh(device,max_edge_L=xi/4,plot=False)

Constructing Voronoi polygons: 100%|██████████| 5730/5730 [00:06<00:00, 840.40it/s] 

num_sites: 5730
num_elements: 11058
min_edge_length: 0.006176786351417163
max_edge_length: 0.02562758848145121
mean_edge_length: 0.014395335339315689
min_area: 2.309249471036603e-05
max_area: 0.0003865282402036166
mean_area: 0.00017452006980802793
coherence_length: 0.1
length_units: um


In [ ]:
for H in simulations_to_be_computed:
	currents=simulations_to_be_computed[H]
	if currents.size == 0:
		continue
	else:
		print(f"simulating H={H}, c={currents}------------------------------------")
		solutions,currents=rs.current_series(device,
						current_range=currents,
						H_field=H,
						solve_time=50
						)
		jj=0
		for solution in solutions:
			voltage=solution.dynamics.voltage()
			time=solution.dynamics.time
			solution.to_hdf5(f'H_solutions/solution_{H}_{currents[jj]}_.hdf5')
			np.save(f'{folder_path}/voltages_{H}_{currents[jj]}_.npy',voltage)
			jj+=1


simulating H=0.0, c=[ 0.          0.50505051  1.01010101  1.51515152  2.02020202  2.52525253
  3.03030303  3.53535354  4.04040404  4.54545455  5.05050505  5.55555556
  6.06060606  6.56565657  7.07070707  7.57575758  8.08080808  8.58585859
  9.09090909  9.5959596  10.1010101  10.60606061 11.11111111 11.61616162
 12.12121212 12.62626263 13.13131313 13.63636364 14.14141414 14.64646465
 15.15151515 15.65656566 16.16161616 16.66666667 17.17171717 17.67676768
 18.18181818 18.68686869 19.19191919 19.6969697  20.2020202  20.70707071
 21.21212121 21.71717172 22.22222222 22.72727273 23.23232323 23.73737374
 24.24242424 24.74747475 25.25252525 25.75757576 26.26262626 26.76767677
 27.27272727 27.77777778 28.28282828 28.78787879 29.29292929 29.7979798
 30.3030303  30.80808081 31.31313131 31.81818182 32.32323232 32.82828283
 33.33333333 33.83838384 34.34343434 34.84848485 35.35353535 35.85858586
 36.36363636 36.86868687 37.37373737 37.87878788 38.38383838 38.88888889
 39.39393939 39.8989899  40.4040

Simulating: 100%|█████████▉| 50/50 [00:12<00:00,  4.03tau/s ]


1/100 done


Simulating: 100%|█████████▉| 50/50 [00:19<00:00,  2.51tau/s ]


2/100 done


...

Simulating: 100%|█████████▉| 50/50 [00:18<00:00,  2.73tau/s ]


5/100 done


Simulating: 100%|█████████▉| 50/50 [00:16<00:00,  3.12tau/s ]


6/100 done


Simulating:  25%|██▍       | 12/50 [00:06<00:09,  3.79tau/s ]

--------------------------------------------------------------------------------


Simulating: 100%|█████████▉| 100/100 [00:21<00:00,  4.66tau/s ]


1/3 done


Simulating: 100%|█████████▉| 100/100 [00:42<00:00,  2.34tau/s ]


2/3 done


...

1/3 done


Simulating: 100%|█████████▉| 100/100 [00:50<00:00,  1.97tau/s ]


2/3 done


Simulating: 100%|█████████▉| 100/100 [00:49<00:00,  2.00tau/s ]


3/3 done
